## Imports

In [ ]:
#Gabriel/Piotrek: Choose index from 1 - 8 for variable PREPROCESSING_OPTIONS[INDEX HERE]
#Run the notebook from top to bottom, for model regression (and urban dictionary basic)
#Save stats; rename it to UD+old_name, and move it to appopriate folder
#LIFEHACK: Make multiple (>2) duplicates of this notebook, open the duplicates in another tab/window, use different indeces, and run them in parallel.

#In this cell, there are various option for the user to freely choose:
#embedding_choice
#PREPROCESSING_CHOICE

import time
from torch.utils.data import DataLoader
import torch.nn as nn
import pandas as pd
import numpy as np
import torch
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models import KeyedVectors
#from gensim.test.utils import common_texts
import gensim.downloader as api
import gensim

#helper function to save models and stats
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pickle


use_drive = True
if use_drive:
  PATH = "/content/drive/MyDrive/CIL 2022/"
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/CIL 2022/
  !ls
else:
  PATH = "./"

#save model
# now you can save it to a file
# with open('filename.pkl', 'wb') as f:
#     pickle.dump(clf, f)

# # and later you can load it
# with open('filename.pkl', 'rb') as f:
#     clf = pickle.load(f)

#1 = stanford glove
#2 = word2vec
#3 = fasttext
#4 = basic urban dictionary embedding
#5 = phrase urban dictionary embedding
def get_embedding_name(embedding_id):
  if embedding_id == 1:
    return "stanford glove"
  elif embedding_id == 2:
    return "word2vec"
  elif embedding_id == 3:
    return "fasttext"
  elif embedding_id == 4:
    return "basic urban dictionary"
  elif embedding_id == 5:
    return "phrase urban dictionary"
  else:
    return "User error: embedding id does not exist."

#Saving trained model
def save_model(model, model_name):
  name_of_file_model = "trained " + model_name + ".pkl"
  with open(name_of_file_model, 'wb') as f:
    pickle.dump(model, f)
  print("Model saved.")

#Saving stats of model to a file
def save_stats_to_file(y_val, y_val_pred, model_name):
  name_of_file_stats = get_embedding_name(embedding_choice) + "+" + model_name + PREPROCESSING_CHOICE + ".txt"
  # name_of_file_model = "trained " + model_name + ".pkl"
  # print("Saving model to file.")
  # with open(name_of_file_model, 'wb') as f:
  #   pickle.dump(model, f)
  print("Saving model relevant stats to file.")
  file_obj = open(name_of_file_stats, "w", encoding="utf8")
  file_obj.write(f'use_drive: {use_drive}\n')
  # file_obj.write(f'LOAD_FROM_SPLITTED_DATASET: {LOAD_FROM_SPLITTED_DATASET}\n')
  # file_obj.write(f'WRITE_SPLITTED_DATASET: {WRITE_SPLITTED_DATASET}\n')
  # file_obj.write(f'REMOVE_DUPLICATES_FROM_TRAINING: {REMOVE_DUPLICATES_FROM_TRAINING}\n')
  file_obj.write(f'PREPROCESSING_CHOICE: {PREPROCESSING_CHOICE}\n')
  file_obj.write(f'embedding_choice: {embedding_choice}\n')
  file_obj.write(f'embedding_name: {get_embedding_name(embedding_choice)}\n')
  file_obj.write(f'dimension_of_embedding: {dimension_of_embedding}\n')
  file_obj.write(f'Model name: {model_name}\n')
  file_obj.write(f'Acc: {accuracy_score(y_val, y_val_pred)}\n')
  file_obj.write(f'Recall: {recall_score(y_val, y_val_pred)}\n')
  file_obj.write(f'Precision: {precision_score(y_val, y_val_pred)}\n')
  file_obj.write(f'F1: {f1_score(y_val, y_val_pred)}\n')
  file_obj.write(f'ROC_AUC: {roc_auc_score(y_val, y_val_pred)}\n')
  file_obj.close()
  print(f'Acc: {accuracy_score(y_val, y_val_pred)}')
  print(f'Recall: {recall_score(y_val, y_val_pred)}')
  print(f'Precision: {precision_score(y_val, y_val_pred)}')
  print(f'F1: {f1_score(y_val, y_val_pred)}')
  print(f'ROC_AUC: {roc_auc_score(y_val, y_val_pred)}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1NSBJKoA0FA5BeDWR8bNo5tPgA5lCU8yv/CIL 2022
'Bert inspiration'
 BoW_v2.ipynb
 code
 data
 Models
 Notes.gdoc
 On_Processed_Bag-of-words_baseline.ipynb
 papers_to_possible_reference
 Preprocessing
 Project_2_Bag-of-words_baseline.ipynb
 __pycache__
 resources
 Results.gsheet
 template.ipynb
'trained Glove_LogisticRegression(C=1, max_iter=1000).pkl'
 trash
 tut1-model.pt
'Word embeddings'
 XGBoost


# Choices

In [ ]:
#Choose embedding below
#1 = stanford glove
#2 = word2vec
#3 = fasttext
#4 = basic urban dictionary embedding
#5 = phrase urban dictionary embedding
embedding_choice = 1

USE_DICTIONARY = True
embedding_non_dict_model = None
embedding_dict = dict()
#SET FLAG "USE_DICTIONARY" to determine if we work with a dictionary or a loaded pretrained model
if (embedding_choice == 2) or (embedding_choice == 3):
  USE_DICTIONARY = False

# HYPERPARAMETERS
# LOAD_FROM_SPLITTED_DATASET = True
# WRITE_SPLITTED_DATASET = True
# REMOVE_DUPLICATES_FROM_TRAINING = False

#option 0 - 8
PREPROCESSING_OPTIONS = [ "raw",
"no-stemming_no-lemmatize_no-stopwords_no-spellcorrect",
"no-stemming_no-lemmatize_with-stopwords_no-spellcorrect",
"no-stemming_no-lemmatize_with-stopwords_with-spellcorrect",
"no-stemming_with-lemmatize_with-stopwords_no-spellcorrect",
"no-stemming_with-lemmatize_with-stopwords_with-spellcorrect",
"with-stemming_no-lemmatize_with-stopwords_no-spellcorrect",
"with-stemming_with-lemmatize_no-stopwords_with-spellcorrect",
"with-stemming_with-lemmatize_with-stopwords_no-spellcorrect" ]
PREPROCESSING_CHOICE = PREPROCESSING_OPTIONS[8] # one from PREPROCESSING_OPTIONS






#Choose embedding

In [ ]:
#From the glove embedding file

#https://github.com/RaRe-Technologies/gensim-data#datasets
#https://kavita-ganesan.com/easily-access-pre-trained-word-embeddings-with-gensim/#.YsGEXexBxE4

#depending on choice, we choose a different file
if embedding_choice == 1:
  #stanford
  #files offers dimension_of_embedding = {25,50,100,200}
  dimension_of_embedding = 200 #can also modify this

  #easy:
  # model_name = "glove-twitter-" + str(dimension_of_embedding)
  # model = api.load(model_name)

  #directly from stanford glove file:
  path_to_embedding_file = "/content/drive/MyDrive/CIL 2022/Word embeddings"
  path_to_embedding_file += "/Glove/glove.twitter.27B." + str(dimension_of_embedding) + "d.txt"

  # FLAG_shaped_already_saved = False
  # shape_of_np_vector = None

  embedding_file_obj = open(path_to_embedding_file, "r", encoding = "utf8")
  for line in embedding_file_obj:
    #Check if line consists only of ASCII characters (no Chinese/Japanse/Arabic, etc. expressions)
    #There were some lines that had non-ASCII characters.
    if line.isascii():
      word_n_vector = line.split()
      embedding_dict[word_n_vector[0]] = np.array(word_n_vector[1:], dtype=np.float64)
      # if not FLAG_shaped_already_saved:
      #   shape_of_np_vector = np.array(word_n_vector[1:], dtype=np.float64).shape
      #   FLAG_shaped_already_saved = True

  embedding_file_obj.close

  #add <UNKNOWN> token to vocabulary
  #choose the 0 vector as the corresponding embedding
  embedding_dict["<UNKNOWN>"] = np.zeros(dimension_of_embedding, dtype=np.float64)
  embedding_dict["<PAD>"] = np.zeros(dimension_of_embedding, dtype=np.float64)
  #embedding_dict["<UNKNOWN>"] = np.zeros(shape_of_np_vector, dtype=np.float64)


elif embedding_choice == 2:
  
#https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html

  #easier:
  dimension_of_embedding = 300
  #choose which embeddings
  # model_name = "word2vec-google-news-300"
  # model_name = "word2vec-ruscorpora-300" #get Russian vocabulary
  
  #IF PRE-TRAINED MODEL NOT STORED, LOAD IT FROM WEB
  model_name = "word2vec-google-news-" + str(dimension_of_embedding)
  embedding_non_dict_model = api.load(model_name) #load pretrained model
  #embedding_non_dict_model.save_word2vec_format('w2v_pre_trained_model.txt', binary=False) #store it #indicate path

  #ELSE LOAD IT FROM FILE
  # pre_trained_model = "/content/drive/MyDrive/CIL 2022/Models/w2v_pre_trained_model.txt"
  # model = KeyedVectors.load_word2vec_format(pre_trained_model, binary=False)

elif embedding_choice == 3:

#https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-auto-examples-tutorials-run-fasttext-py
  
  dimension_of_embedding = 300
  #choose which embeddings
  # model_name = "word2vec-google-news-300"
  # model_name = "word2vec-ruscorpora-300" #get Russian vocabulary

  #IF PRE-TRAINED MODEL NOT STORED, LOAD IT FROM WEB
  model_name = "fasttext-wiki-news-subwords-" + str(dimension_of_embedding)
  embedding_non_dict_model = api.load(model_name) #load pretrained model
  

  #ELSE LOAD IT FROM FILE
  # pre_trained_model = "/content/drive/MyDrive/CIL 2022/Models/fasttext_pre_trained_model.txt"
  # model = KeyedVectors.load_fasttext_format(pre_trained_model, binary=False)
  

elif (embedding_choice == 4) or (embedding_choice == 5):
  
  #see https://aclanthology.org/2020.lrec-1.586.pdf for difference.

  #TODO: figure out what the binaries do
  path_to_embedding_file = None
  if embedding_choice == 4:
    path_to_embedding_file = "/content/drive/MyDrive/CIL 2022/Word embeddings/urbanDictionary/ud_basic.vec"
  else:
    path_to_embedding_file = "/content/drive/MyDrive/CIL 2022/Word embeddings/urbanDictionary/ud_phrase.vec"
  

  embedding_file_obj = open(path_to_embedding_file, "r")
  vocab_size_and_embedding_dim = embedding_file_obj.readline().split()

  vocab_size = int(vocab_size_and_embedding_dim[0])
  dimension_of_embedding = int(vocab_size_and_embedding_dim[1])

  # FLAG_shaped_already_saved = False
  # shape_of_np_vector = None

  for line in embedding_file_obj:
    word_n_vector = line.split()
    embedding_dict[word_n_vector[0]] = np.array(word_n_vector[1:], dtype=np.float64)
    # if not FLAG_shaped_already_saved:
    #   shape_of_np_vector = np.array(word_n_vector[1:], dtype=np.float64).shape
    #   FLAG_shaped_already_saved = True

  embedding_file_obj.close

  #add <UNKNOWN> token to vocabulary
  #choose the 0 vector as the corresponding embedding
  embedding_dict["<UNKNOWN>"] = np.zeros(dimension_of_embedding, dtype=np.float64)
  embedding_dict["<PAD>"] = np.zeros(dimension_of_embedding, dtype=np.float64)
  #embedding_dict["<UNKNOWN>"] = np.zeros(shape_of_np_vector, dtype=np.float64)
else:
  print("Error: embedding not recognized")

#init method

# Tokenize to embeddings

In [ ]:

#https://www.nltk.org/api/nltk.tokenize.html

#feature = average of vectors of tokens occuring in tweet
def transform_to_embeddings(list_of_sentences, embedding_model, embedding_model_type, word_to_ix):

  max_size = 100
  sentences = torch.zeros(len(list_of_sentences), max_size, dtype=torch.long, device='cuda')#to return

    #for each tweet:
    #for each token:
    #get embedding vector
    #compute average
    #store it in a list with corresponding label
  pad_index = word_to_ix.get('<PAD>')

  for line, tweet in enumerate(list_of_sentences):
    tokenized_tweet = tknzr.tokenize(tweet)
    indexed_embeddings = torch.tensor([pad_index]*max_size, dtype=torch.long, device='cuda')
    
    for index, word in enumerate(tokenized_tweet):
  
      word_index = word_to_ix.get(word)
      if word_index == None:
        word_index = word_to_ix.get("<UNKNOWN>")

      indexed_embeddings[index] = word_index
    sentences[line] = indexed_embeddings

  return sentences


#Create training and test set

In [ ]:
def read_file_and_strip(filename):
  lines = []
  with open(filename) as file:
    for line in file:
      lines.append(line.strip())
  return np.array(lines)

def read_data():
  dataset_path = PATH + "data/" + PREPROCESSING_CHOICE + "/"

  train_sentences = read_file_and_strip(dataset_path + "train_sentences.txt")
  train_labels = read_file_and_strip(dataset_path + "train_labels.txt").astype(int)
  val_sentences = read_file_and_strip(dataset_path + "val_sentences.txt")
  val_labels = read_file_and_strip(dataset_path + "val_labels.txt").astype(int)
  
  return train_sentences, train_labels, val_sentences, val_labels

train_sentences, train_labels, val_sentences, val_labels = read_data()
print(len(train_sentences))


embedding_model = None
if USE_DICTIONARY:
  embedding_model_type = "DICT"
  embedding_model = embedding_dict
else:
  embedding_model_type = "NON-DICT"
  embedding_model = embedding_non_dict_model


1967351


## Transform the sentences into tensor vectors of word indexes from the vocab corpus

In [ ]:
# Create a vocabulary corpus
word_to_ix = {word: i for i, word in enumerate(embedding_model)}

train_data = transform_to_embeddings(train_sentences, embedding_model, embedding_model_type, word_to_ix)

val_data = transform_to_embeddings(val_sentences, embedding_model, embedding_model_type, word_to_ix)


In [ ]:

train_labels = torch.tensor(train_labels, dtype=torch.long, device='cuda') 

val_labels = torch.tensor(val_labels, dtype=torch.long, device='cuda') 

## Transform the embeddings into float tensor

In [ ]:
# Transform the embeddings into float tensor with just weights and indexes of 
# those weights representing the words
test = np.zeros((len(embedding_model), dimension_of_embedding))
for i, word in enumerate(embedding_model):
  test[i] = embedding_model[word]

pretrained_embeddings = torch.tensor(test, dtype=torch.float, device='cuda')

#Model = RNN

## Put tensor into a dataset for batching

In [ ]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_id, labels):
        'Initialization'
        self.labels = labels
        self.list_id = list_id

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_id)

  def __getitem__(self, index):
        'Generates one sample of data'


        # Load data and get label
        X = self.list_id[index]
        y = torch.tensor([self.labels[index]], dtype=torch.long, device='cuda')
        return X, y 

In [ ]:
BATCH_SIZE = 64

dataset = Dataset(train_data, train_labels)
my_dataloader = DataLoader(dataset, batch_size=64) # create your dataloader

val_dataset = Dataset(val_data, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=64)

In [ ]:
test = torch.tensor([0, 0, 0], dtype=torch.long).to('cuda')
print(test[0])

tensor(0, device='cuda:0')


In [ ]:
print(train_data.device)

cuda:0


In [ ]:

for index,batch in enumerate(my_dataloader):
  if(index == 1):
    text, l = batch
    print(type(batch))
    print(type(l))
    break


<class 'list'>
<class 'torch.Tensor'>


In [ ]:
#del train_data
#del train_labels

## Model

In [ ]:

class RNN(nn.Module):
    def __init__(self, input, embedding_dim, hidden_dim, output_dim, pad_id):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input, embedding_dim, padding_idx=pad_id)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

## Training method

In [ ]:

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for index, batch in enumerate(iterator):
        
        text, labels = batch

       # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        
        
        optimizer.zero_grad()

        predictions = model(text.T)
        
        loss = criterion(predictions, labels.type(torch.FloatTensor).to('cuda'))
        
        acc = binary_accuracy(predictions, labels.type(torch.FloatTensor).to('cuda'))
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
          
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

## Evaluation method

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for index, batch in enumerate(iterator):
          text, labels = batch

          predictions = model(text.T)
          
          loss = criterion(predictions, labels.type(torch.FloatTensor).to('cuda'))
          
          acc = binary_accuracy(predictions, labels.type(torch.FloatTensor).to('cuda'))
          

          epoch_loss += loss.item()
          epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Run Model

In [ ]:
# One epoch run around 7m on the full dataset
INPUT = len(embedding_dict)
EMBEDDING_DIM = dimension_of_embedding
HIDDEN_DIM = 256
OUTPUT_DIM = 1
PAD_ID = word_to_ix.get('<PAD>')

model = RNN(INPUT, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_ID).to('cuda')

model.embedding.from_pretrained(pretrained_embeddings, freeze=False, padding_idx=PAD_ID)

N_EPOCHS = 10

best_valid_loss = float('inf')

import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division #
    acc = correct.sum() / len(correct)
    return acc

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, my_dataloader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_dataloader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 7m 5s
	Train Loss: 0.693 | Train Acc: 50.10%
	 Val. Loss: 0.693 |  Val. Acc: 50.07%
Epoch: 02 | Epoch Time: 7m 4s
	Train Loss: 0.693 | Train Acc: 50.08%
	 Val. Loss: 0.693 |  Val. Acc: 50.07%
Epoch: 03 | Epoch Time: 7m 5s
	Train Loss: 0.693 | Train Acc: 50.08%
	 Val. Loss: 0.693 |  Val. Acc: 50.07%


KeyboardInterrupt: ignored